# Model-Predictive Path Integral Control Example

In [1]:
%load_ext autoreload
%autoreload 2

## Imports and Parameters

In [8]:
import time
from utils import * # imports integrators and other useful things
from system import * # imports system dynamics and objective
from MPPI import MPPI # imports controller
import matplotlib.pyplot as plt
%matplotlib widget

###################
# System parameters
# x0 = np.array([0,0])
x0 = np.array([0,0,np.pi/2])
del_t = 0.05
w_sz = 15
ov = 1
# dyn = lambda x, u: single_int(x,u) 
dyn = lambda x, u: diff_drive(x,u) 

######################
# Objective parameters
R = np.diag([0.0,0.0])
# xd = np.array([5,2])
# Q = np.diag([1,1])
xd = np.array([5.,2.,0.])
u0 = np.array([0.,0.])
Q = np.diag([1,1,.01])
w1 = 1 # weight given to tracking objective
w2 = 1 # weight given to rattling objective
# obj = lambda x, u: quadratic_objective(x,u,xd,Q,R)
obj = lambda x, u: rattling_objective(x, u, del_t, w1, w2, None, w_sz, ov, xd, Q, R)

#######################
# Controller parameters
noise_mean = np.array([0.0,0.0])
noise_var = np.diag([0.2,0.05])
samp_num = 100
predictive_horizon = 20
simulation_length = 500
Temp = 1.0

## Initialize Controller

In [9]:
controller = MPPI(dyn, obj, x0, u0, K=samp_num, N=predictive_horizon, dt=del_t, lamb=Temp, noise_mu=noise_mean, noise_sigma=noise_var)

## Main Loop

In [10]:
tic = time.time()
xlist = []
ulist = []
costlist = []
for i in range(simulation_length):
    u_star = controller.step(x0)[:,0]
    x = integrate(dyn,x0,u_star,del_t)
    x0 = np.copy(x)
    xlist.append(x0)
    ulist.append(u_star)
    costlist.append(np.mean(controller.cost_total))
xtraj = np.array(xlist).T
utraj = np.array(ulist).T
toc = time.time()
print("Time elapsed: "+str(np.round(toc-tic,2))+"s")

Time elapsed: 66.89s


## Plot

In [15]:
plt.figure()
plt.plot(xtraj[0],xtraj[1])
plt.ylabel('y')
plt.xlabel('x')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
tvec = np.linspace(0,simulation_length*del_t,simulation_length)
plt.figure()
plt.subplot(2,1,1)
plt.plot(tvec,xtraj[0])
plt.plot([tvec[0],tvec[-1]],[xd[0],xd[0]],'k--')
plt.xlim([tvec[0],tvec[-1]])
plt.ylabel('x')
plt.subplot(2,1,2)
plt.plot(tvec,xtraj[1])
plt.plot([tvec[0],tvec[-1]],[xd[1],xd[1]],'k--')
plt.xlim([tvec[0],tvec[-1]])
plt.ylabel('y')
plt.xlabel('Time (s)')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
plt.figure()
rv = rattling_windows(xtraj.T, del_t, w_sz, ov)[0]
tvec = np.linspace(0,len(rv)*del_t,len(rv))
plt.plot(tvec,rv)
plt.ylabel('Rattling')
plt.xlabel('Time (s)')
plt.xlim([tvec[0],tvec[-1]])
plt.grid(True)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
tvec = np.linspace(0,simulation_length*del_t,simulation_length)
plt.figure()
plt.plot(tvec,costlist)
plt.ylabel('Average Cost')
plt.xlabel('Time (s)')
plt.xlim([tvec[0],tvec[-1]])
plt.grid(True)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
tvec = np.linspace(0,simulation_length*del_t,simulation_length)
plt.figure()
c=0
clist=[]
for i in range(utraj.shape[1]):
    c += np.sqrt(utraj[0,i]**2+utraj[0,i]**2)
    clist.append(c)
plt.plot(tvec,clist)
plt.ylabel('Integrated Control Effort')
plt.xlabel('Time (s)')
plt.xlim([tvec[0],tvec[-1]])
plt.grid(True)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
plt.close('all')